In [2]:
from oyv.imports import *

playsound is relying on another python subprocess. Please use `pip install pygobject` if you want playsound to run more efficiently.


#### Mooring coordinate

In [9]:
coord = pickle_load('/home/oyvindl/work/arven_etter_nansen/data/aen_coords_v2.p')

In [10]:
lon0, lat0 = coord['m1'].lon, coord['m1'].lat

## MEAN SEA LEVEL PRESSURE (CARRA)

In [3]:
carra_dir = '/media/oyvindl/ratatosk/data/carra/for_analysis/'
carra_fn = 'carra_nBS_msl_2018_2020.nc'

In [6]:
dc = xr.open_dataset(carra_dir + carra_fn)

In [19]:
xind_m1, yind_m1 = calc.closest_coord_2d(dc.lon.data, dc.lat.data, lon0, lat0)
print('Closest point (%i, %i): %.3fN, %3fE'%(
    xind_m1, yind_m1,  dc.lat[xind_m1, yind_m1], dc.lon[xind_m1, yind_m1],))

Closest point (237, 189): 79.589N, 28.164355E


In [25]:
MSL_m1 = dc.msl[:, xind_m1, yind_m1].data

In [37]:
D = Bunch()
D['MSL'] = MSL_m1*1e-4 # Convert to db..
D['time'] = dc.t.data
pickle_save(D, 'CARRA_MSL_M1_temp.p')

## CTD

In [38]:
mooring = 'm1_1'

In [39]:
    
ddir_rbr =('/media/oyvindl/ratatosk/data/aen_copies/'
           'data_export/for_paper_submission/'
           'processed_netcdf/%s/rbr_ctd/'%mooring)  
if mooring == 'm1_1':
    fn_rbr = ddir_rbr + 'concerto_21m_m1_2018_2019.nc'
elif mooring == 'm1_2':
    fn_rbr = ddir_rbr + 'concerto_20m_m1_2019_2020.nc'
elif mooring == 'm2_1':
    fn_rbr = ddir_rbr + 'concerto_19m_m2_2018_2019.nc'
elif mooring == 'm2_2':
    fn_rbr = ddir_rbr + 'concerto_32m_m2_2019_2020.nc'
rbr = xr.open_dataset(fn_rbr)
t_rbr = date2num(rbr.TIME.data)

In [41]:
rbr

<xarray.Dataset>
Dimensions:  (TIME: 586103)
Coordinates:
  * TIME     (TIME) datetime64[ns] 2018-10-05T07:58:00.000000256 ... 2019-11-...
Data variables:
    TEMP     (TIME) float64 ...
    PSAL     (TIME) float64 ...
    COND     (TIME) float64 ...
    PRES     (TIME) float64 ...
Attributes: (12/69)
    time_coverage_start:             2018-10-05T07:58:00Z
    time_coverage_end:               2019-11-16T08:20:00Z
    date_created:                    2022-04-27T19:11:08Z
    geospatial_bounds:               POLYGON((79.59 28.10, 79.59 28.10))
    geospatial_lon_min:              28.097166666666666
    geospatial_lon_max:              28.097166666666666
    ...                              ...
    id:                              concerto_21m_m1_2018_2019
    site_code:                       M1-1
    instrument:                      RBR Concerto
    instrument_serial_number:        60594
    activity_type:                   In-situ Ocean fixed station(Moored Instr...
    operational_status:              Scientific

In [44]:
D = Bunch()
D['SP'] = rbr.PSAL.data
D['pres'] = rbr.PRES.data
D['TEMP'] = rbr.TEMP.data
D['time'] = t_rbr
pickle_save(D, 'CARRA_CTD_M1_1_temp.p')